# Python `shelve` standard library

https://docs.python.org/3/library/shelve.html

A “shelf” is a persistent, dictionary-like object. The difference with “dbm” databases
is that the values (not the keys!) in a shelf can be essentially arbitrary Python
objects — anything that the pickle module can handle. This includes most class
instances, recursive data types, and objects containing lots of shared sub-objects. The
keys are ordinary strings.

## Takeaway

There are real limitations for `shelve` (see below). This is a fine thing for small
single-user projects but it is not a good idea for any production-level tasks. At that
point you need to level-up and use a real database like SQL Alchemy (for an object
relational mapper) or SQLite (traditional SQL).


In [1]:
import shelve
from pathlib import Path
from pprint import pprint
from astropy.table.table_helpers import simple_table

In [2]:
filepath = Path("random_stuff.db")
if filepath.exists():
    filepath.unlink()

In [3]:
t = simple_table()
t

a,b,c
int64,float64,str1
1,1.0,c
2,2.0,d
3,3.0,e


In [4]:
with shelve.open("random_stuff") as data:
    data["key1"] = "value"
    data["key2"] = {"a": 1, 1: [2, 3, 4]}
    data["table"] = t

In [5]:
def read_shelve(filename):
    with shelve.open(filename) as data:
        out = dict(data)
    return out

In [6]:
data_rt = read_shelve("random_stuff")
data_rt

{'key1': 'value',
 'table': <Table length=3>
   a      b     c  
 int64 float64 str1
 ----- ------- ----
     1     1.0    c
     2     2.0    d
     3     3.0    e,
 'key2': {'a': 1, 1: [2, 3, 4]}}

In [7]:
def add_key_value(filename, key, value):
    with shelve.open(filename) as data:
        data[key] = value

In [8]:
add_key_value("random_stuff", "key3", "value3")
read_shelve("random_stuff")

{'key1': 'value',
 'table': <Table length=3>
   a      b     c  
 int64 float64 str1
 ----- ------- ----
     1     1.0    c
     2     2.0    d
     3     3.0    e,
 'key3': 'value3',
 'key2': {'a': 1, 1: [2, 3, 4]}}

## BEWARE!

- Warning Because the shelve module is backed by pickle, it is insecure to load a shelf
  from an untrusted source. Like with pickle, loading a shelf can execute arbitrary code.

- The shelve module does not support concurrent read/write access to shelved objects. (Multiple simultaneous read accesses are safe.) When a program has a shelf open for writing, no other program should have it open for reading or writing.

- On macOS dbm.ndbm can silently corrupt the database file on updates, **which can cause hard crashes when trying to read from the database.**

This last issue is scary and it seems to be a bit unpredictable, but apparently things
should be OK for key value size below about 4k bytes.

I tried installing `gdbm` which in theory should fix the MacOS issue but this does not
install `_gdbm` which is what is needed for `dbm.gnu` to import. A lot of googling did
not help me.


In [9]:
for size in range(1, 64 * 256, 256):
    print(size)
    key = f"key{size}"
    value = "*" * size
    add_key_value("random_stuff", key, value)
    data_rt = read_shelve("random_stuff")

1
257
513
769
1025
1281
1537
1793
2049
2305
2561
2817
3073
3329
3585
3841
4097
4353
4609
4865
5121
5377
5633
5889
6145
6401
6657
6913


SystemError: Negative size passed to PyBytes_FromStringAndSize